This notebook compares analytical and numerical evaluations of alchemical derivatives of a rotated and translated CH3OH molecule up to the third order in KSDFT.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf
from scipy.spatial.transform import Rotation

pyscf.__version__

'2.2.1'

In [2]:
from FcMole import FcM_like

Rotating and translating CH3OH.

In [3]:
mol_coord = np.array([[-1.27549516,	0.03357710,	0.00000000],
                      [-2.10170657,	-0.87840808,	1.59107621],
                      [-2.10170657,	-0.87840808,	-1.59107621],
                      [-1.76544024,	1.99528228,	0.00000000],
                      [1.41253482,	-0.22423026,	0.00000000],
                      [2.19050133,	1.29874374,	0.00000000]])
# Coordinates in Angstrom
# C	-0.67496302	0.01776824	0.00000000
# H	-1.11217530	-0.46483357	0.84196133
# H	-1.11217530	-0.46483357	-0.84196133
# H	-0.93423081	1.05585799	0.00000000
# O	0.74748129	-0.11865755	0.00000000
# H	1.15916347	0.68726564	0.00000000
rotvec = np.array([1.2 * np.pi, 1.8 * np.pi, 0.8 * np.pi])
rot = Rotation.from_rotvec(rotvec)
mol_coord = rot.apply(mol_coord)
# print(mol_coord)
# print(np.linalg.norm(mol_coord[1]))
for i in range(6):
  mol_coord[i] += [22.4, 8.7, 11.8]
print(mol_coord)

[[21.51976504  8.14268562 12.53661552]
 [22.19523369  6.50972431 13.49736794]
 [19.9104428   7.49931626 11.51581996]
 [20.96614982  9.55062923 13.87808426]
 [23.39549112  9.16151172 10.8826461 ]
 [23.76170114 10.78607451 11.27170604]]


In [4]:
# target_mol = """
# C	-1.27549516	0.03357710	0.00000000
# H	-2.10170657	-0.87840808	1.59107621
# H	-2.10170657	-0.87840808	-1.59107621
# H	-1.76544024	1.99528228	0.00000000
# O	1.41253482	-0.22423026	0.00000000
# H	2.19050133	1.29874374	0.00000000
# """
target_mol = []
atom_symbols = ['C', 'H', 'H', 'H', 'O', 'H']
for i, atom_coords in enumerate(mol_coord):
    # print(atom_coords)
    atom_str = f' {atom_symbols[i]} {atom_coords[0]: .8f} {atom_coords[1]: .8f} {atom_coords[2]: .8f}'
    print(atom_str)
    target_mol.append(atom_str)
print(target_mol)
dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"

 C  21.51976504  8.14268562  12.53661552
 H  22.19523369  6.50972431  13.49736794
 H  19.91044280  7.49931626  11.51581996
 H  20.96614982  9.55062923  13.87808426
 O  23.39549112  9.16151172  10.88264610
 H  23.76170114  10.78607451  11.27170604
[' C  21.51976504  8.14268562  12.53661552', ' H  22.19523369  6.50972431  13.49736794', ' H  19.91044280  7.49931626  11.51581996', ' H  20.96614982  9.55062923  13.87808426', ' O  23.39549112  9.16151172  10.88264610', ' H  23.76170114  10.78607451  11.27170604']


### Analytical evaluation

In [5]:
%load_ext autoreload
from AP_class import APDFT_perturbator as AP

In [6]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RKS(mol_NN)
mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1,2,3,4,5],flag_response_property=True)

converged SCF energy = -115.629840939949


In [7]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [8]:
inefficient_analytical_alchemical_force = np.zeros((6, 6, 3))
for i in range(6):
  inefficient_analytical_alchemical_force[i] = ap_nn.build_inefficient_alchemical_force(i)

Efficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [9]:
efficient_analytical_alchemical_force = np.zeros((6, 6, 3))
for i in range(6):
  efficient_analytical_alchemical_force[i] = ap_nn.build_alchemical_force(i)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [10]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [11]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [12]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs2 = [-fc_param, 0.0, 0.0, 0.0, 0.0, 0.0]
fcs3 = [0.0, fc_param, 0.0, 0.0, 0.0, 0.0]
fcs4 = [0.0, -fc_param, 0.0, 0.0, 0.0, 0.0]
fcs5 = [0.0, 0.0, fc_param, 0.0, 0.0, 0.0]
fcs6 = [0.0, 0.0, -fc_param, 0.0, 0.0, 0.0]
fcs7 = [0.0, 0.0, 0.0, fc_param, 0.0, 0.0]
fcs8 = [0.0, 0.0, 0.0, -fc_param, 0.0, 0.0]
fcs9 = [0.0, 0.0, 0.0, 0.0, fc_param, 0.0]
fcs10 = [0.0, 0.0, 0.0, 0.0, -fc_param, 0.0]
fcs11 = [0.0, 0.0, 0.0, 0.0, 0.0, fc_param]
fcs12 = [0.0, 0.0, 0.0, 0.0, 0.0, -fc_param]

##### Efficient numerical alchemical force

In [13]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
fmol5=FcM_like(mol_NN,fcs=fcs5)
fmol6=FcM_like(mol_NN,fcs=fcs6)
fmol7=FcM_like(mol_NN,fcs=fcs7)
fmol8=FcM_like(mol_NN,fcs=fcs8)
fmol9=FcM_like(mol_NN,fcs=fcs9)
fmol10=FcM_like(mol_NN,fcs=fcs10)
fmol11=FcM_like(mol_NN,fcs=fcs11)
fmol12=FcM_like(mol_NN,fcs=fcs12)
mf=scf.RKS(mol_NN)
mf1=scf.RKS(fmol1)
mf2=scf.RKS(fmol2)
mf3=scf.RKS(fmol3)
mf4=scf.RKS(fmol4)
mf5=scf.RKS(fmol5)
mf6=scf.RKS(fmol6)
mf7=scf.RKS(fmol7)
mf8=scf.RKS(fmol8)
mf9=scf.RKS(fmol9)
mf10=scf.RKS(fmol10)
mf11=scf.RKS(fmol11)
mf12=scf.RKS(fmol12)
mf.xc = dft_functional
mf1.xc = dft_functional
mf2.xc = dft_functional
mf3.xc = dft_functional
mf4.xc = dft_functional
mf5.xc = dft_functional
mf6.xc = dft_functional
mf7.xc = dft_functional
mf8.xc = dft_functional
mf9.xc = dft_functional
mf10.xc = dft_functional
mf11.xc = dft_functional
mf12.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())
e5=mf5.scf(dm0=mf5.init_guess_by_1e())
e6=mf6.scf(dm0=mf6.init_guess_by_1e())
e7=mf7.scf(dm0=mf7.init_guess_by_1e())
e8=mf8.scf(dm0=mf8.init_guess_by_1e())
e9=mf9.scf(dm0=mf9.init_guess_by_1e())
e10=mf10.scf(dm0=mf10.init_guess_by_1e())
e11=mf11.scf(dm0=mf11.init_guess_by_1e())
e12=mf12.scf(dm0=mf12.init_guess_by_1e())

converged SCF energy = -115.629840925878


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -115.644576712914
converged SCF energy = -115.615108121776
converged SCF energy = -115.630966228464
converged SCF energy = -115.628716712783
converged SCF energy = -115.630966228716
converged SCF energy = -115.62871671253
converged SCF energy = -115.630954286492
converged SCF energy = -115.628728652182
converged SCF energy = -115.652201382516
converged SCF energy = -115.60748405551
converged SCF energy = -115.630854673112
converged SCF energy = -115.628828144093


In [14]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()
nuc_grad5 = mf5.nuc_grad_method().kernel()
nuc_grad6 = mf6.nuc_grad_method().kernel()
nuc_grad7 = mf7.nuc_grad_method().kernel()
nuc_grad8 = mf8.nuc_grad_method().kernel()
nuc_grad9 = mf9.nuc_grad_method().kernel()
nuc_grad10 = mf10.nuc_grad_method().kernel()
nuc_grad11 = mf11.nuc_grad_method().kernel()
nuc_grad12 = mf12.nuc_grad_method().kernel()

--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0188774573    -0.0183119558     0.0126138365
1 H    -0.0138195679     0.0132451030    -0.0117982143
2 H     0.0168940584    -0.0000582282     0.0148394572
3 H     0.0077196206    -0.0073763104    -0.0125836040
4 O     0.0181346601     0.0727065647     0.0154543403
5 H    -0.0100152835    -0.0602003741    -0.0185161921
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0188562980    -0.0183051751     0.0125928191
1 H    -0.0138003308     0.0131865336    -0.0117623556
2 H     0.0168352403    -0.0000829902     0.0148076205
3 H     0.0077003271    -0.0073266564    -0.0125365608
4 O     0.0181709070     0.0727421567     0.0154303215
5 H    -0.0100138193    -0.0602090693    -0.0185222229
----------------------------------------------
--------------- RKS gradients ---------------
         x        

In [15]:
efficient_numerical_alchemical_force = np.zeros((6, 6, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)
efficient_numerical_alchemical_force[2] = (nuc_grad5 - nuc_grad6) / (2 * fc_param)
efficient_numerical_alchemical_force[3] = (nuc_grad7 - nuc_grad8) / (2 * fc_param)
efficient_numerical_alchemical_force[4] = (nuc_grad9 - nuc_grad10) / (2 * fc_param)
efficient_numerical_alchemical_force[5] = (nuc_grad11 - nuc_grad12) / (2 * fc_param)

#### Numerical alchemical derivatives

In [16]:
numerical_1st_alchemical_derivative = np.zeros((6))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)
numerical_1st_alchemical_derivative[2] = (e5 - e6) / (2 * fc_param)
numerical_1st_alchemical_derivative[3] = (e7 - e8) / (2 * fc_param)
numerical_1st_alchemical_derivative[4] = (e9 - e10) / (2 * fc_param)
numerical_1st_alchemical_derivative[5] = (e11 - e12) / (2 * fc_param)

#### Numerical alchemical hardness

In [17]:
ap_nn=AP(mf,sites=[0,1,2,3,4,5])
ap_nn1=AP(mf1,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn2=AP(mf2,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn3=AP(mf3,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn4=AP(mf4,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn5=AP(mf5,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn6=AP(mf6,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn7=AP(mf7,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn8=AP(mf8,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn9=AP(mf9,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn10=AP(mf10,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn11=AP(mf11,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
ap_nn12=AP(mf12,sites=[0,1,2,3,4,5],flag_response_property=True,charges_for_center=[6,1,1,1,8,1])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
an5 = ap_nn5.build_gradient()
an6 = ap_nn6.build_gradient()
an7 = ap_nn7.build_gradient()
an8 = ap_nn8.build_gradient()
an9 = ap_nn9.build_gradient()
an10 = ap_nn10.build_gradient()
an11 = ap_nn11.build_gradient()
an12 = ap_nn12.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
ann5 = ap_nn5.build_hessian()
ann6 = ap_nn6.build_hessian()
ann7 = ap_nn7.build_hessian()
ann8 = ap_nn8.build_hessian()
ann9 = ap_nn9.build_hessian()
ann10 = ap_nn10.build_hessian()
ann11 = ap_nn11.build_hessian()
ann12 = ap_nn12.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
dipole5 = ap_nn5.ref_elec_electric_dipole_moment * -1
dipole6 = ap_nn6.ref_elec_electric_dipole_moment * -1
dipole7 = ap_nn7.ref_elec_electric_dipole_moment * -1
dipole8 = ap_nn8.ref_elec_electric_dipole_moment * -1
dipole9 = ap_nn9.ref_elec_electric_dipole_moment * -1
dipole10 = ap_nn10.ref_elec_electric_dipole_moment * -1
dipole11 = ap_nn11.ref_elec_electric_dipole_moment * -1
dipole12 = ap_nn12.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1
pol5 = ap_nn5.ref_elec_electric_polarizability * -1
pol6 = ap_nn6.ref_elec_electric_polarizability * -1
pol7 = ap_nn7.ref_elec_electric_polarizability * -1
pol8 = ap_nn8.ref_elec_electric_polarizability * -1
pol9 = ap_nn9.ref_elec_electric_polarizability * -1
pol10 = ap_nn10.ref_elec_electric_polarizability * -1
pol11 = ap_nn11.ref_elec_electric_polarizability * -1
pol12 = ap_nn12.ref_elec_electric_polarizability * -1

In [18]:
numerical_2nd_alchemical_derivative = np.zeros((6, 6))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)
numerical_2nd_alchemical_derivative[2] = (an5 - an6) / (2 * fc_param)
numerical_2nd_alchemical_derivative[3] = (an7 - an8) / (2 * fc_param)
numerical_2nd_alchemical_derivative[4] = (an9 - an10) / (2 * fc_param)
numerical_2nd_alchemical_derivative[5] = (an11 - an12) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [19]:
numerical_3rd_alchemical_derivative = np.zeros((6, 6, 6))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)
numerical_3rd_alchemical_derivative[2, :, :] = (ann5 - ann6) / (2 * fc_param)
numerical_3rd_alchemical_derivative[3, :, :] = (ann7 - ann8) / (2 * fc_param)
numerical_3rd_alchemical_derivative[4, :, :] = (ann9 - ann10) / (2 * fc_param)
numerical_3rd_alchemical_derivative[5, :, :] = (ann11 - ann12) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [20]:
# mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
# mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
# mf_nn1=scf.RKS(mol_NN1)
# mf_nn2=scf.RKS(mol_NN2)
# mf_nn3=scf.RKS(mol_NN3)
# mf_nn4=scf.RKS(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
# mf_nn1.scf()
# mf_nn2.scf()
# mf_nn3.scf()
# mf_nn4.scf()
# ap_nn1=AP(mf_nn1,sites=[0,1])
# ap_nn2=AP(mf_nn2,sites=[0,1])
# ap_nn3=AP(mf_nn3,sites=[0,1])
# ap_nn4=AP(mf_nn4,sites=[0,1])
# ad1 = ap_nn1.build_gradient()
# ad2 = ap_nn2.build_gradient()
# ad3 = ap_nn3.build_gradient()
# ad4 = ap_nn4.build_gradient()

In [21]:
# inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
# inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
# inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [22]:
numer_elec_electric_dipole_gradient = np.zeros((6, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)
numer_elec_electric_dipole_gradient[2] = (dipole5 - dipole6) / (2 * fc_param)
numer_elec_electric_dipole_gradient[3] = (dipole7 - dipole8) / (2 * fc_param)
numer_elec_electric_dipole_gradient[4] = (dipole9 - dipole10) / (2 * fc_param)
numer_elec_electric_dipole_gradient[5] = (dipole11 - dipole12) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [23]:
numer_elec_electric_pol_gradient = np.zeros((6, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)
numer_elec_electric_pol_gradient[2] = (pol5 - pol6) / (2 * fc_param)
numer_elec_electric_pol_gradient[3] = (pol7 - pol8) / (2 * fc_param)
numer_elec_electric_pol_gradient[4] = (pol9 - pol10) / (2 * fc_param)
numer_elec_electric_pol_gradient[5] = (pol11 - pol12) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [24]:
print(analytical_1st_alchemical_derivative)

[-14.73429549  -1.12475777  -1.12475802  -1.112817   -22.35866273
  -1.01326447]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [25]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[8.30511659e-08 7.44427862e-08 7.36212433e-08 1.55029585e-07
 7.74844757e-07 4.26629629e-08]


$\partial^2E/\partial Z_i\partial Z_j$

In [26]:
print(analytical_2nd_alchemical_derivative)

[[-2.98293488  0.39397392  0.39397416  0.39283337  0.29753093  0.27529668]
 [ 0.39397392 -1.08949115  0.32087867  0.32603015  0.27910378  0.26650407]
 [ 0.39397416  0.32087867 -1.08949071  0.32603002  0.27910405  0.26650438]
 [ 0.39283337  0.32603015  0.32603002 -1.08691796  0.27464635  0.27361312]
 [ 0.29753093  0.27910378  0.27910405  0.27464635 -3.58626911  0.43909166]
 [ 0.27529668  0.26650407  0.26650438  0.27361312  0.43909166 -0.96544979]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [27]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[-3.38645438e-06 -1.47326112e-06 -1.47262626e-06  1.46104895e-06
   2.84473355e-07  1.94189384e-06]
 [ 5.12933201e-06  1.55028473e-06  2.52582127e-07  1.21059683e-06
  -5.31589166e-06 -8.00272133e-07]
 [ 5.15766650e-06  2.57107840e-07  1.55414648e-06  1.21577060e-06
  -5.33519028e-06 -8.05404744e-07]
 [-4.45187453e-06 -1.64566995e-06 -1.64477320e-06  1.48486554e-06
   9.14224820e-07  1.55591877e-06]
 [ 1.18179557e-06  2.39821560e-07  2.39668515e-07 -9.24087349e-07
   3.97121363e-06  2.80894321e-07]
 [-4.10698594e-05 -8.66338363e-06 -8.66090189e-06 -5.42050600e-06
   3.33215201e-05  1.02566648e-05]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [28]:
print(analytical_3rd_alchemical_derivative)

[[[-0.22788707  0.08235853  0.08235857  0.08131596  0.03559971
    0.033763  ]
  [ 0.08235853  0.11526794  0.07214453  0.07344352  0.0454354
    0.04055425]
  [ 0.08235857  0.07214453  0.11526727  0.07344334  0.04543542
    0.0405542 ]
  [ 0.08131596  0.07344352  0.07344334  0.1134919   0.04294746
    0.03649194]
  [ 0.03559971  0.0454354   0.04543542  0.04294746  0.02660791
    0.01943419]
  [ 0.033763    0.04055425  0.0405542   0.03649194  0.01943419
    0.02652867]]

 [[ 0.08235853  0.11526794  0.07214453  0.07344352  0.0454354
    0.04055425]
  [ 0.11526794 -0.82306965  0.06684584  0.06732731  0.0536703
    0.04364444]
  [ 0.07214453  0.06684584  0.06684532  0.03686229  0.02980391
    0.02616275]
  [ 0.07344352  0.06732731  0.03686229  0.06724616  0.02899916
    0.02456287]
  [ 0.0454354   0.0536703   0.02980391  0.02899916  0.02805352
    0.02179832]
  [ 0.04055425  0.04364444  0.02616275  0.02456287  0.02179832
    0.03203328]]

 [[ 0.08235857  0.07214453  0.11526727  0.07344334 

$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [29]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[-4.23679868e-07  2.67890248e-07  2.67321235e-07  2.71887207e-07
    1.72370249e-07  9.99939004e-08]
  [ 2.67890248e-07  2.47864696e-07  4.02769638e-07  2.14670571e-07
    1.94390807e-07  1.15887956e-07]
  [ 2.67321235e-07  4.02769638e-07  2.47492986e-07  2.14327430e-07
    1.94226393e-07  1.15725727e-07]
  [ 2.71887207e-07  2.14670571e-07  2.14327430e-07  8.29898009e-07
    5.01590416e-08  7.65841418e-08]
  [ 1.72370249e-07  1.94390807e-07  1.94226393e-07  5.01590416e-08
   -1.80575969e-08 -9.59696859e-08]
  [ 9.99939004e-08  1.15887956e-07  1.15725727e-07  7.65841418e-08
   -9.59696859e-08  6.80939076e-07]]

 [[ 6.78644540e-07 -2.29669871e-07 -2.47410272e-07 -2.95104124e-07
   -1.53291853e-07 -1.92911752e-07]
  [-2.29669871e-07 -7.72013732e-07 -2.64113509e-07 -3.43223826e-07
   -2.07030300e-07 -2.04518682e-07]
  [-2.47410272e-07 -2.64113509e-07 -6.29213909e-07 -2.53717271e-07
   -1.66382763e-07 -1.75640726e-07]
  [-2.95104124e-07 -3.43223826e-07 -2.53717271e-07 -4.56053360e-07
   -

$\partial^2E/\partial Z_i\partial R_j$

In [30]:
print(efficient_analytical_alchemical_force)

[[[ 0.02116072  0.00678329 -0.02101778]
  [ 0.01924209 -0.05858345  0.03586687]
  [-0.05883187 -0.02476801 -0.0318448 ]
  [-0.01929886  0.04966589  0.04705527]
  [ 0.03625935  0.03559136 -0.02403336]
  [ 0.00146446 -0.00868908 -0.00602798]]

 [[-0.02028339 -0.01456263  0.02729521]
  [ 0.00747734 -0.04712179  0.0260674 ]
  [-0.00938962  0.01107857 -0.00797502]
  [-0.00660248  0.01580577 -0.00611081]
  [ 0.03406549  0.03901858 -0.03403441]
  [-0.00526715 -0.00421938 -0.00524321]]

 [[-0.0302814  -0.0102323   0.01862439]
  [ 0.01237016  0.00165396  0.01089684]
  [-0.04458515 -0.02457374 -0.0190848 ]
  [ 0.01272761  0.0074333   0.01065381]
  [ 0.04680087  0.03350294 -0.02298965]
  [ 0.00296782 -0.00778617  0.00189873]]

 [[-0.02135741 -0.0136617   0.0178034 ]
  [ 0.00480906 -0.01358385 -0.01046369]
  [ 0.00314119 -0.01286148 -0.01191003]
  [-0.01574333  0.04098333  0.0386226 ]
  [ 0.03772144  0.00853834 -0.03923043]
  [-0.00856627 -0.00941469  0.00517535]]

 [[ 0.03684125  0.02509375 -0.02

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [31]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[-9.20581814e-08 -9.88529041e-07 -3.87352603e-07]
  [-1.30098580e-07  4.03803383e-07 -1.99097058e-07]
  [ 3.62561238e-07  1.90413707e-07  2.28987122e-07]
  [-3.96473625e-08  4.39958551e-07  2.65744781e-07]
  [-3.66319571e-07 -1.03190260e-06 -9.32550407e-08]
  [ 2.64073484e-07  9.85210655e-07  1.87561236e-07]]

 [[ 4.28948521e-07  6.96152795e-07 -9.78435644e-07]
  [ 5.06036528e-08 -6.78121321e-07  4.19163873e-07]
  [-2.32348121e-07 -1.68748146e-07  9.50167808e-08]
  [-2.01510075e-07  2.72236100e-07  3.44287752e-07]
  [ 2.06187959e-07  5.38456313e-08 -4.24522673e-08]
  [-2.53088171e-07 -1.71321554e-07  1.61179231e-07]]

 [[ 1.17550778e-06  3.76400915e-07 -3.36032980e-07]
  [-1.52821173e-07 -2.05306428e-07  1.64962797e-07]
  [-6.78894690e-07 -3.62315350e-07 -2.17066852e-07]
  [-1.80604340e-07  2.63962474e-07  3.64547322e-07]
  [ 5.64571644e-08  1.20347415e-07 -1.73121558e-07]
  [-2.14188126e-07 -1.90332879e-07  1.96517293e-07]]

 [[-2.93544566e-07 -1.19535758e-06 -2.58333339e-07]
  [-1.

$\partial^2E/\partial Z_i\partial R_j$

In [32]:
print(inefficient_analytical_alchemical_force)

[[[ 0.02116072  0.00678329 -0.02101778]
  [ 0.01924209 -0.05858345  0.03586686]
  [-0.05883187 -0.02476801 -0.0318448 ]
  [-0.01929886  0.04966589  0.04705527]
  [ 0.03625935  0.03559136 -0.02403336]
  [ 0.00146446 -0.00868908 -0.00602798]]

 [[-0.02028339 -0.01456263  0.02729521]
  [ 0.00747734 -0.04712179  0.0260674 ]
  [-0.00938962  0.01107857 -0.00797502]
  [-0.00660248  0.01580577 -0.00611081]
  [ 0.03406549  0.03901858 -0.03403441]
  [-0.00526715 -0.00421938 -0.00524321]]

 [[-0.0302814  -0.0102323   0.01862439]
  [ 0.01237016  0.00165396  0.01089684]
  [-0.04458515 -0.02457374 -0.0190848 ]
  [ 0.01272761  0.0074333   0.01065381]
  [ 0.04680087  0.03350294 -0.02298965]
  [ 0.00296782 -0.00778617  0.00189873]]

 [[-0.02135741 -0.0136617   0.0178034 ]
  [ 0.00480906 -0.01358385 -0.01046369]
  [ 0.00314119 -0.01286148 -0.01191003]
  [-0.01574333  0.04098333  0.0386226 ]
  [ 0.03772144  0.00853834 -0.03923043]
  [-0.00856627 -0.00941469  0.00517535]]

 [[ 0.03684125  0.02509375 -0.02

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [33]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,efficient}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical,inefficient}$

In [34]:
print(efficient_analytical_alchemical_force - inefficient_analytical_alchemical_force)

[[[-1.06366915e-10  8.06498202e-11  1.62827751e-10]
  [ 2.14643026e-11 -2.37352915e-11  5.56318880e-12]
  [-1.61994862e-11 -7.38689665e-12 -2.70930778e-11]
  [ 1.68036834e-11 -5.96074301e-11 -4.91410801e-11]
  [ 4.97788477e-11 -1.12241771e-10 -1.13405618e-10]
  [ 3.45278389e-11  1.22315005e-10  2.12476391e-11]]

 [[-1.66029523e-10 -1.13160592e-10  4.33522107e-11]
  [-4.20585788e-12  3.57200935e-11 -1.49003032e-11]
  [ 7.35364825e-11  3.23478883e-11  5.05672032e-11]
  [ 9.54659268e-12 -1.24482646e-11 -1.09308881e-11]
  [ 9.42946554e-11  1.25891408e-10 -6.20568041e-11]
  [-7.13287068e-12 -6.83486739e-11 -6.03601544e-12]]

 [[-8.41571257e-11 -1.48635326e-10  1.14374510e-10]
  [-3.30557803e-11  7.85146809e-11 -4.18754476e-11]
  [ 2.93984836e-11  2.11710649e-11  1.42567069e-11]
  [ 3.91842114e-12 -1.00112002e-11 -1.58108665e-11]
  [ 1.08793974e-10  1.19621146e-10 -4.94955604e-11]
  [-2.49012547e-11 -6.06534684e-11 -2.14466309e-11]]

 [[ 1.02446773e-10  8.18725088e-11 -7.73043851e-11]
  [ 1.

$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [35]:
print(elec_electric_dipole_gradient_f)

[[-0.05181399 -0.08055338  0.01951721]
 [ 0.3442823  -1.33121028  0.78395923]
 [-1.30618976 -0.61635595 -0.64745666]
 [-0.52200783  0.91686538  1.059782  ]
 [ 0.64389616  0.22971691 -0.62772051]
 [ 0.89532632  1.48102332 -0.29271739]]


In [36]:
print(elec_electric_dipole_gradient_z)

[[-0.05181399 -0.08055338  0.01951721]
 [ 0.3442823  -1.33121028  0.78395923]
 [-1.30618976 -0.61635595 -0.64745666]
 [-0.52200783  0.91686538  1.059782  ]
 [ 0.64389616  0.22971691 -0.62772051]
 [ 0.89532632  1.48102332 -0.29271739]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [37]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[-1.19532232e-06 -1.16610882e-05 -4.44826753e-06]
 [ 2.25133774e-06  2.21830367e-06 -8.28957451e-06]
 [ 8.34768832e-06 -4.00563789e-07 -3.04248221e-06]
 [-1.20255479e-06 -1.13776981e-05 -4.29580905e-06]
 [-2.98250303e-06  2.40586598e-06  4.63843681e-06]
 [-3.77235414e-05 -3.36433459e-05  2.67092971e-05]]


In [38]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[-1.19515729e-06 -1.16611344e-05 -4.44848165e-06]
 [ 2.25087693e-06  2.21826387e-06 -8.28953577e-06]
 [ 8.34765024e-06 -4.00786669e-07 -3.04207685e-06]
 [-1.20185570e-06 -1.13775484e-05 -4.29654142e-06]
 [-2.98253611e-06  2.40583375e-06  4.63845870e-06]
 [-3.77234521e-05 -3.36432162e-05  2.67092584e-05]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [39]:
print(elec_electric_pol_gradient)

[[[ 4.55355541 -0.77962567  1.56365557]
  [-0.77962567  5.83054934  0.69073029]
  [ 1.56365557  0.69073029  4.78936642]]

 [[ 5.57875594 -1.48355719  1.85214755]
  [-1.48355719  8.19283847 -0.66026941]
  [ 1.85214755 -0.66026941  5.86509444]]

 [[ 7.00032137  0.18380099  2.63542498]
  [ 0.18380099  6.48181824  0.98050526]
  [ 2.63542498  0.98050526  6.15446482]]

 [[ 5.33944189 -0.88310123  0.5625039 ]
  [-0.88310123  7.62090376  1.72289845]
  [ 0.5625039   1.72289845  6.42160196]]

 [[ 6.74793295  0.71186769 -0.68937714]
  [ 0.71186769  6.00256309 -0.74044175]
  [-0.68937714 -0.74044175  6.26685385]]

 [[ 5.11669232  0.53965083 -0.61896997]
  [ 0.53965083  4.72279282 -0.43410218]
  [-0.61896997 -0.43410218  4.84306106]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [40]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[ 1.58612152e-05 -5.12116265e-06  1.10642129e-05]
  [-5.12116265e-06  2.62251069e-05  5.12099314e-06]
  [ 1.10642129e-05  5.12099314e-06  1.74996579e-05]]

 [[ 3.30681943e-06  1.30858547e-05 -1.49638836e-05]
  [ 1.30858547e-05 -1.58152095e-05 -5.70270776e-06]
  [-1.49638836e-05 -5.70270776e-06 -4.63120349e-06]]

 [[-3.61582002e-06  7.43654585e-06 -1.76713222e-05]
  [ 7.43654585e-06 -9.73525532e-06 -1.21485490e-05]
  [-1.76713222e-05 -1.21485490e-05 -4.09273327e-06]]

 [[ 1.96510292e-05 -6.71251296e-06  1.01666893e-05]
  [-6.71251296e-06  2.85233090e-05  6.26383871e-06]
  [ 1.01666893e-05  6.26383871e-06  2.27871449e-05]]

 [[-3.23288279e-05 -4.67781014e-06  7.19615123e-06]
  [-4.67781014e-06 -2.64339240e-05  4.16091781e-06]
  [ 7.19615123e-06  4.16091781e-06 -3.02558864e-05]]

 [[ 5.01268269e-05 -8.34189118e-05  1.32146592e-04]
  [-8.34189118e-05  1.60743193e-04  7.61053159e-05]
  [ 1.32146592e-04  7.61053159e-05  8.65184465e-05]]]
